# NeurIPS 2019 paper viewer
Loads the proceedings from NeurIPS 2019 conference and makes it easy to search with keywords, list selected items and download the papers and supplementary material

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import ipywidgets as widgets
from pprint import pprint
from ipyaggrid import Grid
from IPython.display import display, HTML 
import urllib.request
import shutil
import os

In [3]:
data_file = "neurips2019.csv"
download_path = "neurips_papers"

In [4]:
df = pd.read_csv(data_file, index_col=0)

In [5]:
df.head()

,title,authors,doc_link,abstract,pdf_link,supplemental_link,event_type
0,Multimodal Model-Agnostic Meta-Learning via Ta...,"Risto Vuorio,Shao-Hua Sun,Hexiang Hu,Joseph J....",https://papers.nips.cc/paper/8296-multimodal-m...,Model-agnostic meta-learners aim to acquire me...,https://papers.nips.cc/paper/8296-multimodal-m...,https://papers.nips.cc/paper/8296-multimodal-m...,poster
1,ViLBERT: Pretraining Task-Agnostic Visiolingui...,"Jiasen Lu,Dhruv Batra,Devi Parikh,Stefan Lee",https://papers.nips.cc/paper/8297-vilbert-pret...,We present ViLBERT (short for Vision-and-Langu...,https://papers.nips.cc/paper/8297-vilbert-pret...,https://papers.nips.cc/paper/8297-vilbert-pret...,poster
2,Stochastic Shared Embeddings: Data-driven Regu...,"Liwei Wu,Shuqing Li,Cho-Jui Hsieh,James L. Sha...",https://papers.nips.cc/paper/8298-stochastic-s...,"In deep neural nets, lower level embedding lay...",https://papers.nips.cc/paper/8298-stochastic-s...,https://papers.nips.cc/paper/8298-stochastic-s...,poster
3,Unsupervised Scale-consistent Depth and Ego-mo...,"Jiawang Bian,Zhichao Li,Naiyan Wang,Huangying ...",https://papers.nips.cc/paper/8299-unsupervised...,Recent work has shown that CNN-based depth and...,https://papers.nips.cc/paper/8299-unsupervised...,https://papers.nips.cc/paper/8299-unsupervised...,poster
4,Zero-shot Learning via Simultaneous Generating...,"Hyeonwoo Yu,Beomhee Lee",https://papers.nips.cc/paper/8300-zero-shot-le...,To overcome the absence of training data for u...,https://papers.nips.cc/paper/8300-zero-shot-le...,https://papers.nips.cc/paper/8300-zero-shot-le...,poster


In [6]:
columnDefs = [
    {'headerName': "title", 'field': "title", 'width': 300},
    {'headerName': "authors", 'field': "authors", 'width': 100},
    {'headerName': "doc_link", 'field': "doc_link", 'width': 100, 'hide':True},
    {'headerName': "event_type", 'field': "event_type", 'width': 100, 'hide':True},
    {'headerName': "abstract", 'field': "abstract", 'width': 300},
];

In [7]:
grid_options = {
    'columnDefs': columnDefs,
    'enableSorting': True,
    'enableFilter': True,
    'enableColResize': True,
    'enableRangeSelection': True,
    'defaultColDef': {'sortable': 'true', 'filter': 'true', 'resizable': 'true'},
    'rowSelection': 'multiple',
}

In [8]:
color_grid = Grid(
    width='100%',
    height=300,
    grid_data=df,
    grid_options=grid_options,
    quick_filter=True,
    export_mode='auto',
    export_to_df=True
)

In [9]:
out = widgets.Output(layout={'border': '1px solid black'})
title = widgets.HTML(
    value="",
)

url = widgets.HTML(
    value="",
)

text = widgets.HTML(
    value="",
)

item_list = widgets.HTML(
    value="",
)

download = widgets.Button(
    description='Download Selected',
    disabled=False,
    button_style='',
    tooltip='Download Selected Papers',
    icon='check',
    layout=widgets.Layout(width='30%', height='50px')
)

list_button = widgets.Button(
    description='List Selected',
    disabled=False,
    button_style='',
    tooltip='List selected Items',
    icon='',
    layout=widgets.Layout(width='30%', height='50px')
)


download_path = widgets.Text(
    value = download_path,
    placeholder='Input download path',
    description='Download to:',
    disabled=False
)

In [10]:
def download_to(url, dest_path):
    dest_file = os.path.join(dest_path, os.path.basename(url))
    with urllib.request.urlopen(url) as in_resp, open(dest_file, 'wb') as out_file:
        shutil.copyfileobj(in_resp, out_file)

In [11]:
def on_download_clicked(b):
    with out:
        if not os.path.isdir(download_path.value):
            os.makedirs(download_path.value)
        
        rows = color_grid.grid_data_out.get('rows')
        selected_df = df.iloc[rows.index.get_level_values(0)]
        print(f"Downloading {len(rows)} files to {download_path.value}")
        for i, v in selected_df.iterrows():
            if "Not Found" not in v.pdf_link: 
                print(f"Downloading {v.title} pdf")
                download_to(v.pdf_link, download_path.value)
            
            if "Not Found" not in v.supplemental_link: 
                print(f"Downloading supplemental")
                download_to(v.supplemental_link, download_path.value)

download.on_click(on_download_clicked)

In [12]:
def on_list_clicked(b):
    with out:
        rows = color_grid.grid_data_out.get('rows')
        selected_df = df.iloc[rows.index.get_level_values(0)]
        msgs = []
        for i, row in selected_df.iterrows():
            msgs.append(
                    f"<a href='{row.doc_link}'><h4>{row.title}</h4><h5>({row.event_type})<h5></a> "
                    f"<h5>{row.authors}</h5>"
                    f"<p><a href='{row.pdf_link}'>{row.pdf_link}</a></p>"
                    f"<p><a href='{row.supplemental_link}'> {row.supplemental_link}</a></p>"
                    "<hr><p>  </p><br /> "
            )
        
        item_list.value = "\n".join(msgs)
list_button.on_click(on_list_clicked)

In [13]:
def on_change_data_out(change):
    with out:
        rows = change["new"].get("rows", None)
        if rows is not None:
            selected_df = df.iloc[rows.index.get_level_values(0)]
            download.description=f"Download Selected ({len(rows)})"
            list_button.description=f"List Selected ({len(rows)})"
            text.value = f"<p style='font-size: 130%;'>{selected_df.abstract.iat[0]}<p>"
            title.value =f"<h2>{selected_df.title.iat[0]}<h2>" 
            url.value=f"<a href={selected_df.doc_link.iat[0]}>{selected_df.doc_link.iat[0]}<a>"
    lines = "\n".join([i.get("text", "") for i in out.outputs]).split("\n")
    if len(lines)>10:
        out.clear_output(wait=True)
    
color_grid.observe(on_change_data_out, names='grid_data_out')

In [14]:
buttons = widgets.HBox([download, download_path, list_button])
items = [color_grid, title, url, text, buttons, out, item_list]
widgets.VBox(items)